In [ ]:
import sys
import os
# Go three levels up from current file (i.e., from notebooks/ to project/)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
import numpy as np
import matplotlib.pyplot as plt
import utils.mpl_pubstyle  # noqa: F401
import matplotlib.ticker as ticker
import re, glob

# Plot energy norm for IMEX-RB on 2D nonlinear problem
We show absolute stability of IMEX-RB

In [ ]:

from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset


imex_markers = ['v', '*', 'X', 'd', 'o', 'p', 's', 'h']
colors = ['purple', 'blueviolet', 'magenta', 'hotpink', 'red', 'maroon', 'teal', 'orange']

base_dir = os.path.abspath(os.path.join(os.getcwd(), ''))
problem_name = "Burgers2D"
test_name = "energynorm"
results_base = os.path.join(base_dir, os.pardir, os.pardir, '__RESULTS', problem_name)
convergence_dirs = glob.glob(os.path.join(results_base, f'{test_name}*'))
latest_num = max([int(re.search(rf'{test_name}(\d+)', os.path.basename(d)).group(1)) for d in convergence_dirs])
results_path = os.path.join(results_base, f'{test_name}{latest_num}', 'results.npz')

plots_dir = os.path.join(base_dir, 'plots')
os.makedirs(plots_dir, exist_ok=True)

data = np.load(results_path, allow_pickle=True)
errors_energy = data['errors_energy'].item()
inneriters = data['inneriters'].item()['IMEX-RB'] + 1
Nt = int(data['Nt'])
tvec = np.linspace(0, 1, Nt + 1)
epsilon_values = data['epsilon_values']

linestyles = ['-.', ':', '-', '--']
method_colors = {'BE': 'k', 'FE': 'green'}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

# Top: energy error
for m in ('BE', 'FE'):
    e = errors_energy[m]
    ax1.semilogy(
        tvec[1:], e,
        linestyle='-',
        color=method_colors[m],
        label=f'{m}'
    )

for ie, eps in enumerate(epsilon_values):
    e = errors_energy['IMEX-RB'][ie]
    ax1.semilogy(
        tvec[1:], e,
        marker=imex_markers[ie],
        markersize=6,
        color=colors[ie],
        linestyle='-',
        label=''
    )
axins = inset_axes(
    ax1,
    width="30%",
    height="30%",
    loc='lower right',
    borderpad=2
)
for m in ('BE', 'FE'):
    e = errors_energy[m]
    axins.semilogy(
        tvec[1:], e,
        linestyle='-',
        color=method_colors[m],
    )
for ie, eps in enumerate(epsilon_values):
    e = errors_energy['IMEX-RB'][ie]
    axins.semilogy(
        tvec[1:], e,
        marker=imex_markers[ie],
        markersize=6,
        color=colors[ie],
        linestyle='-',
    )

axins.set_xlim(0.01, 0.05)
axins.set_ylim(2e-5, 5e-5)
axins.tick_params(axis='both', which='both',
                      bottom=False, top=False,
                      left=False, right=False,
                      labelbottom=False, labelleft=False)

mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.5")
ax1.set_ylabel(r'$\mathcal{E}_e(t_n)$', fontsize=15)
ax1.set_xlabel('Time $t_n$', fontsize=17)
ax1.set_ylim(1e-6, 1e2)
ax1.grid(which='major', linestyle='--', linewidth=1)
ax1.tick_params(axis='both', which='major', labelsize=15)

# Bottom: inner iterations
for ie, eps in enumerate(epsilon_values):
    expnt = int(np.log10(eps))
    label = rf'IMEX-RB, $\varepsilon=10^{{{expnt}}}$'
    its = inneriters[ie]
    ax2.plot(
        tvec[1:], its,
        marker=imex_markers[ie],
        markersize=6,
        color=colors[ie],
        linestyle='-',
        label=label
    )

ax2.set_xlabel('Time $t_n$', fontsize=17)
ax2.set_ylabel('Inner iterations', fontsize=15)
ax2.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
ax2.grid(which='major', linestyle='--', linewidth=1)
ax2.tick_params(axis='both', which='major', labelsize=15)

# External legend
fig.legend(
    bbox_to_anchor=(0.5, -0.07),
    loc='lower center',
    ncol=6,
    fontsize=14
)

# plt.tight_layout()
out_file = os.path.join(plots_dir, f'{problem_name}_energyerror_inneriters_eps.pdf')
plt.savefig(out_file, bbox_inches='tight')
plt.show()
plt.close()
